In [2]:
import pandas as pd
import re

pd.set_option('display.max_rows', None)

In [3]:
def remove_quotation(text):
    text = text.strip()
    if text.startswith('"'):
        text = text[1:]

    if text.endswith(','):
        text = text[:-1]
    if text.endswith('"'):
        text = text[:-1]
    return text


def elf_to_df(path):
    friends_dict = {}
    with open(path, 'r', encoding="utf8") as file:
        for line in file.readlines():
            line = re.sub(r'[\[\]]', "", line).strip()
            if ":" in line:
                key, value = line.split(':', 1)
                key = key.replace("\"", "").replace(',', "")
                value = value.replace(r"\u0092", "'")
                value = value.replace(r"\u0085", "...")

                value = remove_quotation(value)
                if key not in friends_dict.keys():
                    friends_dict[key] = []
                friends_dict[key].append(value)
    return pd.DataFrame(friends_dict)

# Emotion lines

In [4]:
# for key, value in friends_dict.items():
#   print(key, value)
#   print('\n')
df = elf_to_df("./raw/EmotionLines/Friends/friends_dev.json")
df1 = elf_to_df("./raw/EmotionLines/Friends/friends_test.json")
df2 = elf_to_df("./raw/EmotionLines/Friends/friends_train.json")


In [9]:
df.head()

,speaker,utterance,emotion,annotation
0,Phoebe,"Oh my God, he's lost it. He's totally lost it.",non-neutral,0002120
1,Monica,What?,surprise,1000130
2,Ross,"Or! Or, we could go to the bank, close our acc...",neutral,3000200
3,Chandler,You're a genius!,joy,0500000
4,Joey,"Aww, man, now we won't be bank buddies!",sadness,0040100


# Empathetic Dialogs

In [4]:
from datasets import load_dataset

In [5]:
ds = load_dataset('empathetic_dialogues')

/home/lucasfl/anaconda3/envs/py38/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Found cached dataset empathetic_dialogues (/home/lucasfl/.cache/huggingface/datasets/empathetic_dialogues/default/0.1.0/09bbeed3882a67db98c73952fb3c1c9a85af83dc78f81454c2454382fd03f6cf)


  0%|          | 0/3 [00:00<?, ?it/s]

In [63]:
val_ds = ds['validation']
ant_ds = val_ds.filter(lambda x:x['context']=='faithful')

Filter:   0%|          | 0/12030 [00:00<?, ? examples/s]

In [64]:
ant_ds["utterance"][0:10]

["My girlfriend hasn't talked to me in a few days and I'm struggling to keep it together but I am at the moment.",
 'Have you tried to call her?',
 "I have_comma_ but she's going through a rough time and likes to seclude herself.",
 'Maybe give her space.',
 'Do you have faithful friends?',
 'Yeah_comma_but very few.',
 'Why is that the case?',
 'Most of my friend have had in the past just wanted to benefit from our relationship_comma_it was not genuine.',
 'I have never cheated on my girlfriend not even once',
 "That's great. I'm trying to get a girlfriend right now :/"]

In [9]:
emp_tag_map = {'anticipating':'happy', 'impressed':'happy',
                'guilty':'sad', 'surprised':'surprised', 
                'furious':'angry', 'excited':'happy', 'sad':'sad',
                'afraid':'sad', 'confident':'happy', 'grateful':'happy',
                'disgusted':'sad', 'jealous':'sad', 'faithful':'happy', 
                'trusting':'happy', 'prepared':'happy', 'joyful':'happy',
                'embarrassed':'sad', 'lonely':'sad', 'ashamed':'sad',
                'devastated':'sad', 'caring':'neutral', 'sentimental':'neutral', 
                'nostalgic':'neutral', 'hopeful':'happy', 'apprehensive':'sad',
                'angry':'angry', 'annoyed':'angry', 'anxious':'sad', 
                'content':'happy', 'terrified':'sad', 'proud':'happy',
                'disappointed':'neutral'}


In [19]:
# val_dict = dict.fromkeys(set(ds['train']["context"]),"")
val_tup = []
for i in ds['validation']:
    val_tup.append({'context': emp_tag_map[i["context"]],
                    'text': i['utterance'].replace('_comma_',', ')})

# GoEmotions